In [41]:
from abc import abstractmethod
from typing import Optional
from typing import Dict
from typing import List
from typing import Union
import numpy as np
from tqdm import tqdm
from omegaconf import OmegaConf
from omegaconf import DictConfig
import hydra
from hydra.core.config_store import ConfigStore
import pytorch_lightning as pl

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import random_split
from torchvision import transforms
import wandb
from datetime import datetime
import sys
import os

In [44]:
sys.path.append("/root/share/pointnet.pytorch/")

In [45]:
from utils.data_utils import dataset_split
from utils.config_utils import flatten_dict
from utils.config_utils import register_config
from utils.config_utils import configure_optimizers_from_cfg
from utils.config_utils import get_loggers
from utils.config_utils import get_callbacks
from utils.custom_math import softmax

In [12]:
class BaseLightningModule(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        self.loss_function = F.nll_loss()
        # self.loss_function = nn.CrossEntropyLoss()

    @abstractmethod
    def forward(self, inputs, target):
        raise NotImplemented()

    def configure_optimizers(self):
        self._optimizer, self._schedulers = configure_optimizers_from_cfg(self.cfg, self)
        return self._optimizer, self._schedulers

    def _forward(self, images, labels, mode: str):

        assert mode in ["train", "val", "test"]

        current_loss = 0.0
        current_corrects = 0

        # get predictions
        #outputs = self(images)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        # get loss(Loss 계산)
        loss = self.loss_function(outputs, labels)
        corrects = torch.sum(preds == labels.data)
        acc = corrects / len(outputs)

        return {
            f"{mode}_loss": loss,
            f"{mode}_acc": acc,
        }

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logs = self._forward(images, labels, mode="train")
        self.log_dict(logs)
        logs["loss"] = logs["train_loss"]
        cul_lr = self._optimizer.param_groups[0]["lr"] if self._schedulers is None else self._schedulers[0].get_last_lr()[0]
        wandb.log({"learning_rate": cul_lr})

        return logs

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logs = self._forward(images, labels, mode="val")
        self.log_dict(logs)
        logs["loss"] = logs["val_loss"]
        return logs

    def test_step(self, batch, batch_idx):
        images, labels = batch
        logs = self._forward(images, labels, mode="test")
        self.log_dict(logs)
        logs["loss"] = logs["test_loss"]
        return logs

In [13]:
class PointNetCls(BaseLightningModule):
    def __init__(self, cfg: DictConfig):
        BaseLightningModule.__init__(self, cfg=cfg)
        
        self.feat = PointNetfeat(**cfg)
        self.fc1 = nn.Linear(cfg.model.cls.layer1.in_feature, cfg.model.cls.layer1.out_feature)
        self.fc2 = nn.Linear(cfg.model.cls.layer2.in_feature, cfg.model.cls.layer2.out_feature)
        self.fc3 = nn.Linear(cfg.model.cls.layer3.in_feature, cfg.model.cls.layer3.k)
        self.dropout = nn.Dropout(cfg.model.cls.dropout_prob)
        self.bn1 = nn.BatchNorm1d(cfg.model.cls.layer1.out_feature)
        self.bn2 = nn.BatchNorm1d(cfg.model.cls.layer2.out_feature)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat

In [14]:
class PointNetDenseCls(BaseLightningModule):
    def __init__(self, cfg: DictConfig):
        BaseLightningModule.__init__(self, cfg=cfg)
        
        self.k = cfg.model.densecls.k
        self.feat = PointNetfeat(**cfg)
        self.conv1 = torch.nn.Conv1d(
            cfg.model.densecls.layer1.conv1d_in_channel,
            cfg.model.densecls.layer1.conv1d_out_channel,
            cfg.model.densecls.layer1.conv1d_kernel_size
        )
        self.conv2 = torch.nn.Conv1d(
            cfg.model.densecls.layer2.conv1d_in_channel,
            cfg.model.densecls.layer2.conv1d_out_channel,
            cfg.model.densecls.layer2.conv1d_kernel_size
        )
        self.conv3 = torch.nn.Conv1d(
            cfg.model.densecls.layer3.conv1d_in_channel,
            cfg.model.densecls.layer3.conv1d_out_channel,
            cfg.model.densecls.layer3.conv1d_kernel_size
        )
        self.conv4 = torch.nn.Conv1d(
            cfg.model.densecls.layer4.conv1d_in_channel,
            self.k,
            cfg.model.densecls.layer3.conv1d_kernel_size
        )
        self.bn1 = nn.BatchNorm1d(cfg.model.densecls.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.densecls.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.densecls.layer3.conv1d_out_channel)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat

In [15]:
class PointNetfeat(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(
            cfg.model.feat.layer1.conv1d_in_channel,
            cfg.model.feat.layer1.conv1d_out_channel,
            cfg.model.feat.layer1.conv1d_kernel_size
        )
        self.conv2 = torch.nn.Conv1d(
            cfg.model.feat.layer2.conv1d_in_channel,
            cfg.model.feat.layer2.conv1d_out_channel,
            cfg.model.feat.layer2.conv1d_kernel_size
        )
        self.conv3 = torch.nn.Conv1d(
            cfg.model.feat.layer3.conv1d_in_channel,
            cfg.model.feat.layer3.conv1d_out_channel,
            cfg.model.feat.layer3.conv1d_kernel_size
        )
        self.bn1 = nn.BatchNorm1d(cfg.model.feat.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.feat.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.feat.layer3.conv1d_out_channel)
        self.global_feat = cfg.model.global_feat
        self.feature_transform = cfg.model.feature_transform
        if self.feature_transform:
            self.fstn = STNkd(**cfg)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

In [16]:
class STN3d(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        conv1 = torch.nn.Conv1d(
            cfg.model.stn.layer1.conv1d_in_channel,
            cfg.model.stn.layer1.conv1d_out_channel,
            cfg.model.stn.layer1.conv1d_kernel_size
        )

        conv2 = torch.nn.Conv1d(
            cfg.model.stn.layer2.conv1d_in_channel,
            cfg.model.stn.layer2.conv1d_out_channel,
            cfg.model.stn.layer2.conv1d_kernel_size
        )
    
        conv3 = torch.nn.Conv1d(
            cfg.model.stn.layer3.conv1d_in_channel,
            cfg.model.stn.layer3.conv1d_out_channel,
            cfg.model.stn.layer3.conv1d_kernel_size
        )
    
        fc1 = nn.Linear(cfg.model.stn.layer4.fc1_in_features, cfg.model.stn.layer4.fc1_out_features)
        fc2 = nn.Linear(cfg.model.stn.layer5.fc2_in_features, cfg.model.stn.layer5.fc2_out_features)
        fc3 = nn.Linear(cfg.model.stn.layer6.fc3_in_features, cfg.model.stn.layer6.fc3_out_features)
    
        bn1 = nn.BatchNorm1d(cfg.model.stn.layer1.conv1d_out_channel)
        bn2 = nn.BatchNorm1d(cfg.model.stn.layer2.conv1d_out_channel)
        bn3 = nn.BatchNorm1d(cfg.model.stn.layer3.conv1d_out_channel)
        bn4 = nn.BatchNorm1d(cfg.model.stn.layer4.fc1_out_features)
        bn5 = nn.BatchNorm1d(cfg.model.stn.layer5.fc3_out_features)
        
        relu = nn.ReLU()


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x

In [17]:
class STNkd(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.k = model.stn.k
        
        conv1 = torch.nn.Conv1d(
            self.k,
            cfg.model.stn.layer1.conv1d_out_channel,
            cfg.model.stn.layer1.conv1d_kernel_size
        )

        conv2 = torch.nn.Conv1d(
            cfg.model.stn.layer2.conv1d_in_channel,
            cfg.model.stn.layer2.conv1d_out_channel,
            cfg.model.stn.layer2.conv1d_kernel_size
        )
    
        conv3 = torch.nn.Conv1d(
            cfg.model.stn.layer3.conv1d_in_channel,
            cfg.model.stn.layer3.conv1d_out_channel,
            cfg.model.stn.layer3.conv1d_kernel_size
        )
    
        fc1 = nn.Linear(cfg.model.stn.layer4.fc1_in_features, cfg.model.stn.layer4.fc1_out_features)
        fc2 = nn.Linear(cfg.model.stn.layer5.fc2_in_features, cfg.model.stn.layer5.fc2_out_features)
        fc3 = nn.Linear(cfg.model.stn.layer6.fc3_in_features, self.k*self.k)
    
        bn1 = nn.BatchNorm1d(cfg.model.stn.layer1.conv1d_out_channel)
        bn2 = nn.BatchNorm1d(cfg.model.stn.layer2.conv1d_out_channel)
        bn3 = nn.BatchNorm1d(cfg.model.stn.layer3.conv1d_out_channel)
        bn4 = nn.BatchNorm1d(cfg.model.stn.layer4.fc1_out_features)
        bn5 = nn.BatchNorm1d(cfg.model.stn.layer5.fc3_out_features)
        
        relu = nn.ReLU()


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [49]:
# model configs
model_pointnet_cfg = {
    "name": "PointNet",
    "cls": {
        "layer1":{
            "in_feature": 1024,
            "out_feature": 512,
        },
        "layer2":{
            "in_feature": 512,
            "out_feature": 256,
        },
        "layer3":{
            "in_feature": 256,
            "out_feature": 26, ################## 
        },
        "dropout" : 0.3,

    },
    "densecls": {
        "k" : 26, ##################
        "layer1": {
            "conv1d_in_channel": 1088,
            "conv1d_out_channel": 512,
            "conv1d_kernel_size": 1,
        },
        "layer2": {
            "conv1d_in_channel": 512,
            "conv1d_out_channel": 256,
            "conv1d_kernel_size": 1,
        },
        "layer3": {
            "conv1d_in_channel": 256,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
        "layer4":{
            "conv1d_in_channel": 128,
            "conv1d_kernel_size": 1,
        },
    },
    "feat":{
        "layer1": {
            "conv1d_in_channel": 3,
            "conv1d_out_channel": 64,
            "conv1d_kernel_size": 1,
        },
         "layer2": {
            "conv1d_in_channel": 64,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
         "layer3": {
            "conv1d_in_channel": 128,
            "conv1d_out_channel": 1024,
            "conv1d_kernel_size": 1,
        },
        "global_feat": True,
        "feature_transform": True,
    },
    "stn": {
        "k": 64, 
        "layer1": {
          "conv1d_in_channel": 3,
          "conv1d_out_channel": 64,
          "conv1d_kernel_size": 1,
        },
        "layer2": {
          "conv1d_in_channel": 64,
          "conv1d_out_channel": 128,
          "conv1d_kernel_size": 1,
        },
        "layer3": {
          "conv1d_in_channel": 128,
          "conv1d_out_channel": 1024,
          "conv1d_kernel_size": 1,
        },
        "layer4": {
          "fc2_in_features": 1024,
          "fc2_out_features": 512,
        },
        "layer5": {
          "fc2_in_features": 512,
          "fc2_out_features": 256,
        },
        "layer6": {
          "fc3_in_features": 256,
          "fc3_out_features": 9,
        }  
    }
}

        
# optimizer configs
opt_cfg = {
    "optimizers": [
        {
            "name": "Adam",
            "kwargs": {
                "lr": 1e-3,
            }
        }
    ],
    "lr_schedulers": [
        {
            "name": "LinearWarmupLR",
            "kwargs": {
                "warmup_end_steps": 1000
            }
        },
    ]
}

data_shapenet_cfg = {
    "name" : "shapenet",
    "data_root" : os.path.join(os.getcwd(), "data"),
    "num_points" : 2500
}

_merged_cfg_presets = {
    "PointNet": {
        "opt": opt_cfg,
        "data": data_shapenet_cfg,
        "model": model_pointnet_cfg,
    },
}

# clear config instance first
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialize & make config
## select mode here ##
# .................. #
hydra.initialize(config_path=None)
cfg = hydra.compose("PointNet")

# override some cfg
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{cfg.model.name}-{cfg.data.name}"

# Define other train configs & log_configs
# Merge configs into one & register it to Hydra.

project_root_dir = os.path.join(
    os.getcwd(), "runs", "pointnet-runs"
)
save_dir = os.path.join(project_root_dir, run_name)
run_root_dir = os.path.join(project_root_dir, run_name)

train_cfg = {
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 32,
    "train_val_split": [0.9, 0.1],
    "run_root_dir": run_root_dir,
    "trainer_kwargs": {
        "accelerator": "auto",
        "max_epochs": 50,
        "val_check_interval": 1.0,
        "log_every_n_steps": 100,
    }
}

# logger config
log_cfg = {
    "loggers": {
        "WandbLogger": {
            "project": "pointNet",
            "name": run_name,
            "tags": ["fastcampus_de_en_translate_tutorials"],
            "save_dir": run_root_dir,
        },
    },
    "callbacks": {
        "ModelCheckpoint": {
            "save_top_k": 3,
            "monitor": "val_loss",
            "mode": "min",
            "verbose": True,
            "dirpath": os.path.join(run_root_dir, "weights"),
            "filename": "{epoch}-{val_loss:.3f}",
        },
        "EarlyStopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 3,
            "verbose": True,
        }
    }
}

# unlock config & set train_cfg & log_cfg
OmegaConf.set_struct(cfg, False)
cfg.train = train_cfg
cfg.log = log_cfg

# lock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

opt:
  optimizers:
  - name: Adam
    kwargs:
      lr: 0.001
  lr_schedulers:
  - name: LinearWarmupLR
    kwargs:
      warmup_end_steps: 1000
data:
  name: shapenet
  data_root: /root/share/pointnet.pytorch/pointnet/data
  num_points: 2500
model:
  name: PointNet
  cls:
    layer1:
      in_feature: 1024
      out_feature: 512
    layer2:
      in_feature: 512
      out_feature: 256
    layer3:
      in_feature: 256
      out_feature: 26
    dropout: 0.3
  densecls:
    k: 26
    layer1:
      conv1d_in_channel: 1088
      conv1d_out_channel: 512
      conv1d_kernel_size: 1
    layer2:
      conv1d_in_channel: 512
      conv1d_out_channel: 256
      conv1d_kernel_size: 1
    layer3:
      conv1d_in_channel: 256
      conv1d_out_channel: 128
      conv1d_kernel_size: 1
    layer4:
      conv1d_in_channel: 128
      conv1d_kernel_size: 1
  feat:
    layer1:
      conv1d_in_channel: 3
      conv1d_out_channel: 64
      conv1d_kernel_size: 1
    layer2:
      conv1d_in_channel: 64
     

/tmp/ipykernel_93054/4203120565.py:137: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path=None)


In [ ]:
data_root = cfg.data.data_root

# 천처리 부분 (preprocessing) & 데이터 셋 정의.
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            cfg.model.feature.normalize.mean,
            cfg.model.feature.normalize.std,
        ) # mean, # std
    ]
)

dataset = ShapeNetDataset(
    root=data_root, 
    classification=True, 
    npoints=cfg.data.num_points
)

test_dataset = ShapeNetDataset(
    root=data_root,
    classification=True,
    split='test',
    npoints=cfg.data.num_point,
    data_augmentation=False
)
        

datasets = dataset_split(dataset, split=cfg.train.train_val_split)

train_dataset = datasets["train"]
val_dataset = datasets["val"]

train_batch_size = cfg.train.train_batch_size
val_batch_size = cfg.train.val_batch_size
test_batch_size = cfg.train.test_batch_size

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=0
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=True, num_workers=0
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=0
)
